# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 


### Creating the Dataframe

### Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import lxml.html as lh
import requests
from bs4 import BeautifulSoup

### Copying the webpage

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page= requests.get(wikipedia_link)

### Extract the Table from the page

In [3]:
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [4]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postal Code

2 Borough

3 Neighborhood



In [5]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

### Creating a Dataframe

In [6]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

### Preprocessing

In [7]:
df.columns=[str(i).rstrip() for  i in df.columns]
for i in df.columns:
    df[i]=df[i].str.rstrip()

In [8]:
df=df[df['Borough']!='Not assigned']
df=df.drop([180],axis=0)


In [9]:
for i in df['Neighborhood']:
    if i=='Not assigned':
        i=df['Borough']

# 

In [10]:
df.shape[0]

103

# ---------------------------------Part 1 End-------------------------------------

# Part 2


## Importing the Coordinates for all the Neighborhoods

In [11]:
coord=pd.read_csv('/home/kuroosan/Downloads/Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the DataFrames to get the final one

In [12]:
final=pd.merge(df,coord,left_on='Postal Code',right_on='Postal Code',how='inner')
final

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# -------------------------------------Part 2 End-----------------------------------------


# Part 3

## Mapping Toronto

In [13]:
import folium

map_toronto=folium.Map(location=[43.6532,-79.3832],zoom_start=10)
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Slicing the Dataframe for boroughs which have the word Toronto in it

In [15]:
import re
l=[i for i in final['Borough'].unique() if re.search('Toronto',i)]


In [16]:
tor=final[final['Borough'].str.contains('Toronto')].reset_index()

In [17]:
tor=tor.drop('index',axis=1)

## Mapping the Boroughs with word Toronto in it

In [18]:
client_id='HARZ0QJYBSLQGDRIIWILH1OBHCYTN10P4ECIRXZGREJRR2GM'
client_secret='3PO42P3ODMOMN0N0HHXMURFA10VOTSKFZWIDMWF5JJKGQ1WR'
VERSION = '20180605'

In [19]:
map_toronto=folium.Map(location=[43.6532,-79.3832],zoom_start=10)
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

# Exploring the Neighborhoods

In [20]:
tor.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

In [21]:
thelat=tor.loc[0,'Latitude']
thelon=tor.loc[0,'Longitude']
thename=tor.loc[0,'Neighborhood']

In [22]:
limit=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id,client_secret,VERSION,thelat,thelon,radius,limit)

### Getting the Neighborhood data using FourSquare

In [23]:
import json
from pandas.io.json import json_normalize
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee86459dc59643ef4334ab3'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Corktown Common,Park,43.655618,-79.356211
7,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503


# Exploring all the Neighborhoods in the Sliced Dataframe

## Function to iterate through all the Neighborhoods

In [26]:
def getnearbyvenues(names,latitudes,longitudes,radius=500):
    
    venueslist=[]
    count=0
    for name,lat,long in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id,client_secret,VERSION,lat,long,radius,limit)
        result=requests.get(url).json()['response']['groups'][0]['items']
        count+=1
        print(count, ' Done')
        venueslist.append([(name,lat,long,r['venue']['name'],r['venue']['location']['lat'],r['venue']['location']['lng'],r['venue']['categories'][0]['name']) for r in result])
    
    return(venueslist)    

In [27]:
listt=getnearbyvenues(names=tor['Neighborhood'],latitudes=tor['Latitude'],longitudes=tor['Longitude'],radius=500)

1  Done
2  Done
3  Done
4  Done
5  Done
6  Done
7  Done
8  Done
9  Done
10  Done
11  Done
12  Done
13  Done
14  Done
15  Done
16  Done
17  Done
18  Done
19  Done
20  Done
21  Done
22  Done
23  Done
24  Done
25  Done
26  Done
27  Done
28  Done
29  Done
30  Done
31  Done
32  Done
33  Done
34  Done
35  Done
36  Done
37  Done
38  Done
39  Done


## Creating a Dataframe

In [28]:
df=pd.DataFrame([item for ven in listt for item in ven])
df.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
1613,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1614,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1615,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1616,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher


## Number of venues in each Neighborhood

In [29]:
df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,31,31,31,31,31,31


In [30]:
print('There are {} uniques categories.'.format(len(df['Venue Category'].unique())))

There are 237 uniques categories.


## Onehot encoding used and the mean of each category

In [32]:
one=pd.get_dummies(df['Venue Category'])
one['Neighborhood']=df['Neighborhood']
cols=[one.columns[-1]]+list(one.columns[:-1])
one=one[cols]
onemean=one.groupby('Neighborhood').mean().reset_index()
onemean

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012821,0.012821,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.032258,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Creating a Dataframe with 10 most visited venues in each Neighborhood

In [33]:
lol=pd.DataFrame()
for i in onemean['Neighborhood']:
    temp=onemean[onemean['Neighborhood']==i].T.reset_index()
    temp=temp.loc[1:,:]
    temp.columns=['Venue','freq']
    temp.freq=temp.freq.astype(float)
    temp['Neighborhood']=i
    temp=temp.sort_values(by='freq',ascending=False)
    temp=temp.head(10)
    l=[]
    l=['1st','2nd','3rd']
    for i in range(4,11):
        l.append(str(i)+'th')
    temp['order']=l
    lol=lol.append(temp)

In [34]:
cols=['Neighborhood','Venue','order']
lol=lol[cols]
lol=lol.pivot(index='Neighborhood',columns='order')
lol.head()

Venue  \
order                                                             10th   
Neighborhood                                                             
Berczy Park                                                     Bistro   
Brockton, Parkdale Village, Exhibition Place                 Nightclub   
Business reply mail Processing Centre, South Ce...         Pizza Place   
CN Tower, King and Spadina, Railway Lands, Harb...    Sculpture Garden   
Central Bay Street                                  Chinese Restaurant   

                                                                     \
order                                                           1st   
Neighborhood                                                          
Berczy Park                                             Coffee Shop   
Brockton, Parkdale Village, Exhibition Place                   Café   
Business reply mail Processing Centre, South Ce...      Yoga Studio   
CN Tower, King and Spadina, Railway Lands, Harb...  Airport Service   
Central Bay Street                                      Coffee Shop   

                                                                    \
order                                                          2nd   
Neighborhood                                                         
Berczy Park                                           Cocktail Bar   
Brockton, Parkdale Village, Exhibition Place           Coffee Shop   
Business reply mail Processing Centre, South Ce...   Auto Workshop   
CN Tower, King and Spadina, Railway Lands, Harb...  Airport Lounge   
Central Bay Street                                            Café   

                                                                          \
order                                                                3rd   
Neighborhood                                                               
Berczy Park                                                   Restaurant   
Brockton, Parkdale Village, Exhibition Place              Breakfast Spot   
Business reply mail Processing Centre, South Ce...  Gym / Fitness Center   
CN Tower, King and Spadina, Railway Lands, Harb...      Airport Terminal   
Central Bay Street                                    Italian Restaurant   

                                                                         \
order                                                               4th   
Neighborhood                                                              
Berczy Park                                          Seafood Restaurant   
Brockton, Parkdale Village, Exhibition Place                     Bakery   
Business reply mail Processing Centre, South Ce...        Garden Center   
CN Tower, King and Spadina, Railway Lands, Harb...                Plane   
Central Bay Street                                  Japanese Restaurant   

                                                                    \
order                                                          5th   
Neighborhood                                                         
Berczy Park                                            Cheese Shop   
Brockton, Parkdale Village, Exhibition Place                   Gym   
Business reply mail Processing Centre, South Ce...          Garden   
CN Tower, King and Spadina, Railway Lands, Harb...        Boutique   
Central Bay Street                                  Sandwich Place   

                                                                        \
order                                                              6th   
Neighborhood                                                             
Berczy Park                                                     Bakery   
Brockton, Parkdale Village, Exhibition Place        Italian Restaurant   
Business reply mail Processing Centre, South Ce...  Light Rail Station   
CN Tower, King and Spadina, Railway Lands, Harb...         Coffee Shop   
Central Bay Street                                   

In [35]:
l=[]
for i,j in enumerate(lol.columns):
    l.append(j[1])
lol.columns=l
l

['10th', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th']

In [36]:
l=['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th','10th']

In [37]:
lol=lol[l]
lol=lol.reset_index()
lol

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bakery,Café,Beer Bar,Fish Market,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Gym,Italian Restaurant,Performing Arts Venue,Convenience Store,Restaurant,Nightclub
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Boutique,Coffee Shop,Bar,Boat or Ferry,Rental Car Location,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Salad Place,Department Store,Bubble Tea Shop,Burger Joint,Chinese Restaurant
5,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Baby Store,Diner,Restaurant,Nightclub,Italian Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Pub,Men's Store,Mediterranean Restaurant,Yoga Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Pizza Place,Park,Gourmet Shop
9,Davisville North,Gym,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Park,Pizza Place,Sandwich Place,Gym / Fitness Center,Trail


# Clustering

In [38]:

from sklearn.cluster import KMeans

cluster=onemean.drop('Neighborhood',1)
km=KMeans(n_clusters=5,random_state=0).fit(cluster)
km.labels_[0:10] 

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
lol.insert(0,'Cluster Labels',km.labels_)
lol

,Cluster Labels,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bakery,Café,Beer Bar,Fish Market,Bistro
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Gym,Italian Restaurant,Performing Arts Venue,Convenience Store,Restaurant,Nightclub
2,1,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Boutique,Coffee Shop,Bar,Boat or Ferry,Rental Car Location,Sculpture Garden
4,0,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Salad Place,Department Store,Bubble Tea Shop,Burger Joint,Chinese Restaurant
5,0,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Baby Store,Diner,Restaurant,Nightclub,Italian Restaurant
6,0,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Café,Pub,Men's Store,Mediterranean Restaurant,Yoga Studio
7,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant
8,0,Davisville,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Pizza Place,Park,Gourmet Shop
9,0,Davisville North,Gym,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Park,Pizza Place,Sandwich Place,Gym / Fitness Center,Trail


## Merging the frequent venues with the coordinates

In [44]:
merged=tor
merged=merged.join(lol.set_index('Neighborhood'),on='Neighborhood')
merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
18,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,3,Park,Bus Line,Swim School,Yoga Studio,Monument / Landmark,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store


## Mapping the Clusters

In [43]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

x=np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat,lon,p,clu in zip(merged['Latitude'],merged['Longitude'],merged['Neighborhood'],merged['Cluster Labels']):
    label=folium.Popup(str(p)+' Cluster '+str(clu),parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[clu-1],fill=True,fill_color=rainbow[clu-1],fill_opacity=0.7).add_to(map_clusters)

map_clusters

# ---------------------------------------Part 3 End-------------------------------------